# Training Analysis - Learning Dynamics and Stability

Analyze training behavior, stability mechanisms, and convergence

In [1]:
import numpy as np
import sys
sys.path.insert(0, '..')

from src.training.training_utils import GCSTrainer, early_stopping, warm_start_optimization
from src.visualization.plotly_dashboard import TrainingDashboard

print("✓ Training modules loaded")

✓ Training modules loaded


## 1. Gradient Clipping - Preventing Exploding Gradients

In [2]:
# Create trainer with gradient clipping
trainer = GCSTrainer(
    learning_rate=0.001,
    max_gradient_norm=1.0,
    patience=10
)

print("Trainer Configuration:")
print(f"  Learning Rate: {trainer.learning_rate}")
print(f"  Max Gradient Norm: {trainer.max_gradient_norm}")
print(f"  Patience (early stopping): {trainer.patience}")

INFO:src.training.training_utils:GCSTrainer initialized


Trainer Configuration:
  Learning Rate: 0.001
  Max Gradient Norm: 1.0
  Patience (early stopping): 10


In [3]:
# Demonstrate gradient clipping
print("\nGradient Clipping Demo:")
print("=" * 50)

test_gradients = [
    np.array([0.1, 0.2, 0.3]),      # Normal
    np.array([5.0, 10.0, 15.0]),    # Large
    np.array([100.0, 200.0, 300.0]) # Exploding
]

for i, grads in enumerate(test_gradients):
    norm_before = np.linalg.norm(grads)
    clipped = trainer.gradient_clipping(grads.copy())
    norm_after = np.linalg.norm(clipped)
    
    print(f"\nGradient Set {i+1}:")
    print(f"  Before: norm = {norm_before:.4f}")
    print(f"  After:  norm = {norm_after:.4f}")
    print(f"  Clipped: {clipped}")


Gradient Clipping Demo:

Gradient Set 1:
  Before: norm = 0.3742
  After:  norm = 0.3742
  Clipped: [0.1 0.2 0.3]

Gradient Set 2:
  Before: norm = 18.7083
  After:  norm = 1.0000
  Clipped: [0.26726124 0.53452248 0.80178373]

Gradient Set 3:
  Before: norm = 374.1657
  After:  norm = 1.0000
  Clipped: [0.26726124 0.53452248 0.80178373]


## 2. Early Stopping - Preventing Overfitting

In [4]:
# Simulate training with early stopping
print("Early Stopping Demo:")
print("=" * 50)

# Generate loss curve
losses = []
for epoch in range(30):
    if epoch < 10:
        loss = 1.0 - epoch * 0.08 + np.random.randn() * 0.05
    else:
        loss = 0.3 + np.random.randn() * 0.08  # Plateau
    losses.append(loss)

print(f"\nTraining Losses: {[f'{l:.4f}' for l in losses[:15]]}")

# Check early stopping
stopped = early_stopping(losses, patience=5)
print(f"\nEarly Stopping Triggered: {stopped}")
print(f"Reason: Loss plateaued after {len(losses)} epochs")

Early Stopping Demo:

Training Losses: ['0.9966', '0.8944', '0.7387', '0.7901', '0.6840', '0.6624', '0.5753', '0.3676', '0.3460', '0.2561', '0.2703', '0.4362', '0.1570', '0.2474', '0.3499']

Early Stopping Triggered: True
Reason: Loss plateaued after 30 epochs


In [5]:
# Using trainer's early stopping
trainer2 = GCSTrainer(patience=5)

print("\nUsing Trainer Early Stopping:")
print("=" * 50)

for epoch, loss in enumerate(losses):
    if trainer2.check_early_stopping(loss):
        print(f"\n✓ Early stopping at epoch {epoch}")
        print(f"  Best loss: {trainer2.best_loss:.4f}")
        print(f"  Final loss: {loss:.4f}")
        break
    
    if epoch % 5 == 0:
        trend = trainer2.get_loss_trend(window=5)
        print(f"Epoch {epoch}: Loss = {loss:.4f}, Trend = {trend}")

INFO:src.training.training_utils:GCSTrainer initialized
INFO:src.training.training_utils:Early stopping triggered after 5 epochs



Using Trainer Early Stopping:
Epoch 0: Loss = 0.9966, Trend = insufficient_data
Epoch 5: Loss = 0.6624, Trend = improving
Epoch 10: Loss = 0.2703, Trend = improving
Epoch 15: Loss = 0.3070, Trend = improving

✓ Early stopping at epoch 17
  Best loss: 0.1570
  Final loss: 0.3028


## 3. Warm Start Optimization

In [6]:
# Define a simple objective function
def rosenbrock(x):
    """Rosenbrock function for testing optimization"""
    return (1 - x[0])**2 + 100 * (x[1] - x[0]**2)**2

def objective_wrapper(initial_point):
    """Wrapper for warm start optimization"""
    try:
        # Simple gradient descent for demo
        x = initial_point.copy()
        learning_rate = 0.001
        
        for _ in range(100):
            # Approximate gradient
            eps = 1e-5
            grad = np.array([
                (rosenbrock(x + np.array([eps, 0])) - rosenbrock(x - np.array([eps, 0]))) / (2*eps),
                (rosenbrock(x + np.array([0, eps])) - rosenbrock(x - np.array([0, eps]))) / (2*eps)
            ])
            x = x - learning_rate * grad
        
        return {
            'solution': x,
            'cost': rosenbrock(x),
            'converged': True
        }
    except:
        return {'cost': float('inf'), 'converged': False}

def initial_point_generator():
    return np.random.uniform(-2, 2, 2)

print("Warm Start Optimization Demo:")
print("=" * 50)

# Run warm start
solution = warm_start_optimization(
    objective_wrapper,
    num_restarts=3,
    initial_point_generator=initial_point_generator
)

print(f"\nBest solution cost: {solution['cost']:.6f}")
print(f"Optimal should be near (1, 1) with cost ~0")

Warm Start Optimization Demo:


INFO:src.training.training_utils:Restart 0: Cost=0.659280



Best solution cost: 0.659280
Optimal should be near (1, 1) with cost ~0


## 4. Full Training Loop with Monitoring

In [7]:
# Complete training loop
trainer = GCSTrainer(
    learning_rate=0.001,
    max_gradient_norm=1.0,
    patience=15
)
dashboard = TrainingDashboard()

print("Full Training Loop:")
print("=" * 50)

max_epochs = 100
for epoch in range(max_epochs):
    # Simulate training step
    # Loss decreases then plateaus
    if epoch < 30:
        base_loss = 100 * np.exp(-epoch / 15)
    else:
        base_loss = 10 + 0.01 * (epoch - 30)
    
    loss = base_loss + np.random.randn() * 0.5
    reward = 50 * (1 - np.exp(-epoch/25))
    success_rate = min(1.0, epoch / 40)
    
    # Simulate gradients
    gradients = np.random.randn(10)
    
    # Apply stability mechanisms
    clipped_grads = trainer.gradient_clipping(gradients)
    
    # Update dashboard
    dashboard.update_metrics(
        episode=epoch,
        reward=reward,
        loss=loss,
        success_rate=success_rate
    )
    
    # Log progress
    trainer.log_progress(epoch, loss, reward=reward, success_rate=success_rate)
    
    # Check early stopping
    if trainer.check_early_stopping(loss):
        print(f"\n✓ Stopped at epoch {epoch}")
        break

print(f"\nTraining completed after {epoch+1} epochs")

INFO:src.training.training_utils:GCSTrainer initialized
INFO:src.visualization.plotly_dashboard:Dashboard initialized
INFO:src.training.training_utils:Epoch    0: Loss=99.999808, reward=0.000000, success_rate=0.000000
INFO:src.training.training_utils:Epoch   10: Loss=51.758702, reward=16.483998, success_rate=0.250000
INFO:src.training.training_utils:Epoch   20: Loss=25.819769, reward=27.533552, success_rate=0.500000
INFO:src.training.training_utils:Epoch   30: Loss=8.650215, reward=34.940289, success_rate=0.750000
INFO:src.training.training_utils:Epoch   40: Loss=9.274163, reward=39.905174, success_rate=1.000000
INFO:src.training.training_utils:Early stopping triggered after 15 epochs


Full Training Loop:

✓ Stopped at epoch 45

Training completed after 46 epochs


In [8]:
# Display summary
dashboard.print_summary()

# Create dashboard
fig = dashboard.create_dashboard(filename="training_analysis.html")
fig.show()

print("✓ Analysis complete - dashboard saved to training_analysis.html")


TRAINING METRICS SUMMARY

LOSS
  Mean:   32.607919
  Std:    26.074674
  Min:     8.650215
  Max:    99.999808
  Count: 46

REWARD
  Mean:   26.681566
  Std:    12.054249
  Min:     0.000000
  Max:    41.735056
  Count: 46

SUCCESS_RATE
  Mean:    0.554348
  Std:     0.319702
  Min:     0.000000
  Max:     1.000000
  Count: 46

PATH_LENGTH
  Mean:    0.000000
  Std:     0.000000
  Min:     0.000000
  Max:     0.000000
  Count: 46

PLANNING_TIME
  Mean:    0.000000
  Std:     0.000000
  Min:     0.000000
  Max:     0.000000
  Count: 46




INFO:src.visualization.plotly_dashboard:Dashboard saved to training_analysis.html


✓ Dashboard saved to training_analysis.html


✓ Analysis complete - dashboard saved to training_analysis.html


## Key Takeaways

1. **Gradient Clipping**: Prevents NaN by limiting gradient magnitude
2. **Early Stopping**: Stops training when loss plateaus
3. **Warm Starting**: Multiple initializations improve solution quality
4. **Monitoring**: Track metrics in real-time during training
5. **Stability**: Combines all mechanisms for robust training

## Common Issues and Solutions

| Issue | Cause | Solution |
|-------|-------|----------|
| NaN Loss | Exploding gradients | Increase `max_gradient_norm` clipping |
| No Progress | Bad initialization | Use warm start with more restarts |
| Overfitting | Training too long | Lower `patience` for earlier stopping |
| Slow Convergence | Learning rate too low | Adaptive learning rate |
| Oscillations | Learning rate too high | Use learning rate decay |